# Direct Inversion: Optimization-Free Text-Driven Real Image Editing with Diffusion Models
@author: Adham Elarabawy // aelarabawy@berkeley.edu

## Imports

In [ ]:
from PIL import Image
import torch
import PIL
import numpy as np
from torch import autocast
from diffusers import StableDiffusionPipeline
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler
from pipeline.direct_inversion_pipeline import DirectInversionPipeline
from tqdm import tqdm
from pipeline.util import image_grid, encode, decode

## Base Configuration

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"
remove_safety = False

## Load Direct Inversion Pipeline

In [ ]:
pipe = DirectInversionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)

if remove_safety:
  pipe.safety_checker = lambda images, clip_input: (images, False)
  
pipe = pipe.to(device)

## Run Direct Inversion Pipeline

In [ ]:
# Fill this out:
in_img = Image.open("images/shoes.png").convert("RGB").resize((512,512))
prompt = "black dress shoes" 
inversion_prompt = ""
num_images = 1
guidance_scale = 3          # in range [0, 10]
noise_merge_lambda = 0.5    # in range [0, 1]
num_inference_steps = 100   # in range [0, 1000] *Recommend keeping at 100*
num_inversion_steps = 100   # in range [0, 1000] *Recommend keeping at 100*
direct_injection = True     # toggle if we start from random noise [False] or if we start from inverted noise [True]
stochastic_injection = True # toggle if we continually inject inverted noise
num_stochastic_injection_steps = num_inference_steps # how many steps we inject inverted noise for
stochastic_injection_exp_method = True # toggle if we want to use the scaled noises predicted during inversion for stochastic injection

log = []
output = []
prompts = [ prompt ] * num_images
with autocast("cuda"):
    out = pipe(prompts, 
                guidance_scale=guidance_scale, 
                num_inference_steps=num_inference_steps, 
                input_image=in_img, 
                direct_injection=direct_injection,
                stochastic_injection=stochastic_injection, 
                inversion_steps=num_inversion_steps,
                noise_merge_lambda=noise_merge_lambda,
                num_stochastic_injection_steps=num_stochastic_injection_steps,
                inversion_prompt=inversion_prompt,
                debug_print=True,
                stochastic_injection_exp_method=stochastic_injection_exp_method)
    images = out[0]["sample"]
    # DEBUG [Not enabled for release]
    # noises = out[1]
    # cleans = out[2]
    images.insert(0, in_img)

## Visualize Results

In [ ]:
image_grid(images, 1, len(images))